In [ ]:
import pandas as pd

polls = pd.read_csv('data/house_polls_merged.csv')
elections = pd.read_csv('data/1976-2022-house.csv')

print(polls.columns)
print(elections.columns)
print(elections.shape)

elections_2018_2022 = elections[elections['year'] >= 2018]
print(elections_2018_2022.shape)
print(set(elections_2018_2022['year']))
elections_2018_2022.to_csv('data/2018-2022-house.csv')

Index(['poll_id', 'pollster_id', 'pollster', 'sponsor_ids', 'sponsors',
       'display_name', 'pollster_rating_id', 'pollster_rating_name',
       'numeric_grade', 'pollscore', 'methodology', 'transparency_score',
       'state', 'start_date', 'end_date', 'sponsor_candidate_id',
       'sponsor_candidate', 'sponsor_candidate_party', 'endorsed_candidate_id',
       'endorsed_candidate_name', 'endorsed_candidate_party', 'question_id',
       'sample_size', 'population', 'subpopulation', 'population_full',
       'tracking', 'created_at', 'notes', 'url', 'url_article', 'url_topline',
       'url_crosstab', 'source', 'internal', 'partisan', 'race_id', 'cycle',
       'office_type', 'seat_number', 'seat_name', 'election_date', 'stage',
       'nationwide_batch', 'ranked_choice_reallocated', 'ranked_choice_round',
       'hypothetical', 'party', 'answer', 'candidate_id', 'candidate_name',
       'pct'],
      dtype='object')
Index(['year', 'state', 'state_po', 'state_fips', 'state_cen', 'st